In [1]:

from dotenv import load_dotenv
from utils.db_handler import DatabaseHandler
import pandas as pd
import spacy
from collections import Counter
from typing import List

# creating database handler instance
my_db_handler = DatabaseHandler()
table_name = "optigame_products"

In [2]:
# returning data from the database
df = my_db_handler.retrieve_all_from_table(table_name)
df.head(10)

,reviews_count,price,rating,id,description,asin,title,sales_volume
0,3c943db5-ad7e-45da-85a5-2363584335b1,B07MV4NN5Z,"Big Potato The Chameleon, Award-Winning Board ...",15.99,4.7,2K+ bought in past month,,7063
1,0ce92bc9-af3e-4e03-a362-75dfdac41f84,B09NYD15ZH,Skillmatics Rapid Rumble - Fast-Paced Board Ga...,19.97,4.6,1K+ bought in past month,,3147
2,44ae72ff-b3cb-45b2-b38f-a814e6a26d3e,B076HK9H7Z,"Sorry! Kids Board Game, Family Board Games for...",7.49,4.8,10K+ bought in past month,"Slide, collide and score to win the Sorry! gam...",31332
3,99f3cd2b-7e42-4abd-a76f-32ae6855834a,B00D7OAOYI,"Hasbro Gaming Trouble Kids Board Game, Pop-o-M...",8.99,4.7,10K+ bought in past month,Product Description The Trouble game is racing...,37102
4,ceb3ba0f-3bdf-45eb-8d93-637369b6d069,B00000DMF5,Hasbro Gaming Candy Land Kingdom of Sweet Adve...,10.49,4.8,10K+ bought in past month,Product Description If you loved playing the C...,35049
5,4ce2ba59-d635-4021-b5ae-037f4c2fc4fa,B00D8STBHY,"Hasbro Gaming Connect 4 Classic Grid,4 in a Ro...",8.99,4.8,9K+ bought in past month,Product Description Go 4 the win! The Connect ...,78198
6,4030f2ef-db79-41c2-8bd0-5cb202da0c38,B0B5HLZ8T4,"Monopoly Game, Family Board Games for 2 to 6 P...",19.92,4.8,6K+ bought in past month,"Buy, sell, dream, and scheme the way to riches...",3579
7,3edbf10f-2cb6-4f54-ac45-46398d8096fb,B00ABA0ZOA,Jenga Game | The Original Wood Block Game with...,10.40,4.8,#1 Top Rated,Product Description Want a game experience tha...,100759
8,57234401-010f-42a9-b773-e98ef129cd4d,B077Z1R28P,Taco Cat Goat Cheese Pizza,9.84,4.8,10K+ bought in past month,This is the social card game that is going to ...,51819
9,25e4faba-3811-4e1e-87b6-90fdae232236,B07P6MZPK3,Mattel Games UNO Card Game in a Collectible St...,10.99,4.8,10K+ bought in past month,UNO is the classic family card game that's eas...,71007


### Defining Helper Functions

In [3]:


def extract_common_noun_phrases_with_numbers(nlp, df: pd.DataFrame, title_column: str = "title") -> pd.DataFrame:
    """
    Extracts common nouns and combined noun phrases (including number-noun phrases) from the title column of a DataFrame.

    Args:
        nlp: The spaCy language model.
        df (pd.DataFrame): The input DataFrame containing a 'title' column.
        title_column (str): The name of the column containing text data.

    Returns:
        pd.DataFrame: A DataFrame with an additional 'common_noun_phrases' column containing lists of common nouns and noun phrases.
    """
    # Ensure the title column exists
    if title_column not in df.columns:
        raise ValueError(f"Column '{title_column}' not found in DataFrame.")

    # Function to extract common nouns and noun phrases from a single text
    def get_common_noun_phrases(text):
        ext = text.lower()
        doc = nlp(text)
        noun_phrases = []
        current_phrase = []

        for i, token in enumerate(doc):
            if token.pos_ == "NOUN":  # Check if the token is a noun
                # If the previous token is a number, include it in the phrase
                if i > 0 and doc[i - 1].pos_ == "NUM":
                    current_phrase.append(doc[i - 1].text)
                current_phrase.append(token.text)
            else:
                if current_phrase:  # If we have a phrase, join it and add to the list
                    noun_phrases.append(" ".join(current_phrase))
                    current_phrase = []

        # Add the last phrase if it exists
        if current_phrase:
            noun_phrases.append(" ".join(current_phrase))

        return noun_phrases

    # Apply the function to the title column
    df["common_noun_phrases"] = df[title_column].apply(lambda x: get_common_noun_phrases(str(x)))
    return df


def lemmatize_common_noun_phrases(nlp, df: pd.DataFrame, common_noun_phrases_column: str = "common_noun_phrases") -> pd.DataFrame:
    """
    Lemmatizes the words in the 'common_noun_phrases' column of a DataFrame.

    Args:
        nlp: The spaCy language model.
        df (pd.DataFrame): The input DataFrame containing a 'common_noun_phrases' column.
        common_noun_phrases_column (str): The name of the column containing lists of common noun phrases.

    Returns:
        pd.DataFrame: A DataFrame with the 'common_noun_phrases' column lemmatized.
    """
    # Ensure the common_noun_phrases column exists
    if common_noun_phrases_column not in df.columns:
        raise ValueError(f"Column '{common_noun_phrases_column}' not found in DataFrame.")

    # Function to lemmatize a list of noun phrases
    def lemmatize_phrases(phrases_list):
        lemmatized_phrases = []
        for phrase in phrases_list:
            doc = nlp(phrase)
            lemmatized_phrase = " ".join([token.lemma_ for token in doc])
            lemmatized_phrases.append(lemmatized_phrase)
        return lemmatized_phrases

    # Apply the lemmatization function to the common_noun_phrases column
    df[common_noun_phrases_column] = df[common_noun_phrases_column].apply(lemmatize_phrases)
    return df

def eliminate_shorter_subtags(df: pd.DataFrame, common_noun_phrases_column: str = "common_noun_phrases") -> pd.DataFrame:
    """
    Eliminates shorter subtags from the 'common_noun_phrases' column if a longer parent tag exists.

    Args:
        df (pd.DataFrame): The input DataFrame containing a 'common_noun_phrases' column.
        common_noun_phrases_column (str): The name of the column containing lists of common noun phrases.

    Returns:
        pd.DataFrame: A DataFrame with the 'common_noun_phrases' column updated to remove shorter subtags.
    """
    # Ensure the common_noun_phrases column exists
    if common_noun_phrases_column not in df.columns:
        raise ValueError(f"Column '{common_noun_phrases_column}' not found in DataFrame.")

    # Function to remove shorter subtags
    def filter_subtags(phrases_list):
        # Sort phrases by length in descending order
        phrases_list = sorted(phrases_list, key=len, reverse=True)
        filtered_phrases = []

        for phrase in phrases_list:
            # Add the phrase if it's not a substring of any already added phrase
            if not any(phrase in longer_phrase for longer_phrase in filtered_phrases):
                filtered_phrases.append(phrase)

        return filtered_phrases

    # Apply the filtering function to the common_noun_phrases column
    df[common_noun_phrases_column] = df[common_noun_phrases_column].apply(filter_subtags)
    return df

def get_top_common_nouns(df: pd.DataFrame, common_nouns_column: str = "common_nouns", top_x: int = 10) -> List[str]:
    """
    Returns a list of the top x most common nouns from the 'common_nouns' column.

    Args:
        df (pd.DataFrame): The input DataFrame containing a 'common_nouns' column.
        common_nouns_column (str): The name of the column containing lists of common nouns.
        top_x (int): The number of top common nouns to return.

    Returns:
        List[str]: A list of the top x most common nouns.
    """
    # Ensure the common_nouns column exists
    if common_nouns_column not in df.columns:
        raise ValueError(f"Column '{common_nouns_column}' not found in DataFrame.")

    # Flatten the list of common nouns across all rows
    all_nouns = [noun for nouns_list in df[common_nouns_column] for noun in nouns_list]

    # Count the frequency of each noun
    noun_counts = Counter(all_nouns)

    # Get the top x most common nouns
    top_common_nouns = [noun for noun, _ in noun_counts.most_common(top_x)]

    return top_common_nouns


def filter_tags_with_substring(df: pd.DataFrame, common_noun_phrases_column: str = "common_noun_phrases", substring: str = "game") -> List[str]:
    """
    Filters and returns all tags from the 'common_noun_phrases' column that contain a specific substring.

    Args:
        df (pd.DataFrame): The input DataFrame containing a 'common_noun_phrases' column.
        common_noun_phrases_column (str): The name of the column containing lists of common noun phrases.
        substring (str): The substring to search for in the tags.

    Returns:
        List[str]: A list of tags containing the specified substring.
    """
    # Ensure the common_noun_phrases column exists
    if common_noun_phrases_column not in df.columns:
        raise ValueError(f"Column '{common_noun_phrases_column}' not found in DataFrame.")

    # Flatten the list of tags across all rows and filter by substring
    tags_with_substring = [
        tag for phrases_list in df[common_noun_phrases_column]
        for tag in phrases_list if substring in tag
    ]

    return tags_with_substring

def filter_and_order_tags_by_frequency(df: pd.DataFrame, common_noun_phrases_column: str = "common_noun_phrases", substring: str = "game") -> List[tuple]:
    """
    Filters tags containing a specific substring (not at the start of the string) and orders them by their frequency of occurrence.

    Args:
        df (pd.DataFrame): The input DataFrame containing a 'common_noun_phrases' column.
        common_noun_phrases_column (str): The name of the column containing lists of common noun phrases.
        substring (str): The substring to filter tags by.

    Returns:
        List[tuple]: A list of tuples where each tuple contains a tag and its frequency, sorted by frequency in descending order.
    """
    # Ensure the common_noun_phrases column exists
    if common_noun_phrases_column not in df.columns:
        raise ValueError(f"Column '{common_noun_phrases_column}' not found in DataFrame.")

    # Flatten the list of tags across all rows and filter by substring
    filtered_tags = [
        tag for phrases_list in df[common_noun_phrases_column]
        for tag in phrases_list if substring in tag and not tag.startswith(substring)
    ]

    # Count the frequency of each filtered tag
    tag_counts = Counter(filtered_tags)

    # Sort tags by frequency in descending order
    sorted_tags = tag_counts.most_common()

    return sorted_tags

def add_game_tags_column(df: pd.DataFrame, most_frequent_games: List[tuple], common_noun_phrases_column: str = "common_noun_phrases") -> pd.DataFrame:
    """
    Adds a new column 'game_tags' to the DataFrame containing tags that match between the row's
    'common_noun_phrases' column and the most_frequent_games list.

    Args:
        df (pd.DataFrame): The input DataFrame containing a 'common_noun_phrases' column.
        most_frequent_games (List[tuple]): A list of tuples where each tuple contains a tag and its frequency.
        common_noun_phrases_column (str): The name of the column containing lists of common noun phrases.

    Returns:
        pd.DataFrame: The updated DataFrame with a new 'game_tags' column.
    """
    # Extract the tags from the most_frequent_games list
    frequent_game_tags = {tag for tag, _ in most_frequent_games}

    # Function to find matching tags for each row
    def find_matching_tags(phrases_list):
        return [tag for tag in phrases_list if tag in frequent_game_tags]

    # Apply the function to the common_noun_phrases column
    df["game_tags"] = df[common_noun_phrases_column].apply(find_matching_tags)
    return df


### Running Spacy Model

In [4]:
column_of_interest = "title"

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Extracting common nouns from the DataFrame
df_with_nouns = extract_common_noun_phrases_with_numbers(nlp, df, "title")

# Lemantizing extracted common nouns
df_with_lemmatized_nouns = lemmatize_common_noun_phrases(nlp, df_with_nouns, "common_noun_phrases")
df_with_lemmatized_nouns.head()

,reviews_count,price,rating,id,description,asin,title,sales_volume,common_noun_phrases
0,3c943db5-ad7e-45da-85a5-2363584335b1,B07MV4NN5Z,"Big Potato The Chameleon, Award-Winning Board ...",15.99,4.7,2K+ bought in past month,,7063,[]
1,0ce92bc9-af3e-4e03-a362-75dfdac41f84,B09NYD15ZH,Skillmatics Rapid Rumble - Fast-Paced Board Ga...,19.97,4.6,1K+ bought in past month,,3147,[]
2,44ae72ff-b3cb-45b2-b38f-a814e6a26d3e,B076HK9H7Z,"Sorry! Kids Board Game, Family Board Games for...",7.49,4.8,10K+ bought in past month,"Slide, collide and score to win the Sorry! gam...",31332,"[slide, collide, score, game, player, opponent..."
3,99f3cd2b-7e42-4abd-a76f-32ae6855834a,B00D7OAOYI,"Hasbro Gaming Trouble Kids Board Game, Pop-o-M...",8.99,4.7,10K+ bought in past month,Product Description The Trouble game is racing...,37102,"[product, game, peg, fun, excitement, player, ..."
4,ceb3ba0f-3bdf-45eb-8d93-637369b6d069,B00000DMF5,Hasbro Gaming Candy Land Kingdom of Sweet Adve...,10.49,4.8,10K+ bought in past month,Product Description If you loved playing the C...,35049,"[product description, board game, child, exper..."


In [5]:
top_x_nouns = get_top_common_nouns(df_with_lemmatized_nouns, common_nouns_column="common_noun_phrases", top_x=30)
print("Top 10 common nouns:", top_x_nouns)

Top 10 common nouns: ['game', 'player', 'card', 'trademark', 'kid', 'world', 'time', 'games', 'age', 'fun', 'friend', 'character', 'opponent', 'family', 'way', 'adventure', 'turn', 'strategy', 'board', 'team', 'board game', 'adult', 'card game', 'color', 'tile', 'point', 'role', 'challenge', 'experience', 'gift']


### Now Eliminating Shorter Phrases

In [6]:
df_with_collapsed_tags = eliminate_shorter_subtags(df_with_lemmatized_nouns, "common_noun_phrases")
top_x_nouns_2 = get_top_common_nouns(df_with_collapsed_tags, common_nouns_column="common_noun_phrases", top_x=30)
print("Top 10 common nouns:", top_x_nouns_2)

Top 10 common nouns: ['player', 'trademark', 'world', 'time', 'friend', 'age', 'fun', 'turn', 'way', 'opponent', 'character', 'game', 'games', 'logo', 'card', 'choice', 'adult', 'gameplay', 'gift', 'role', 'kid', 'strategy', '4 player', 'challenge', 'adventure', 'family', 'product', 'card game', 'rule', 'game night']


In [7]:
game_tags = filter_tags_with_substring(df_with_collapsed_tags, "common_noun_phrases", "game")
print("Tags containing 'game':", game_tags)

Tags containing 'game': ['family board game', 'kid board game', 'game night', 'games', 'trouble gameplay', 'kid board game', 'family game', 'board games', 'game closet', 'game piece', 'game night', 'game board', 'party game', 'board game', 'game board', 'game play', 'gameboard', 'game card', 'player board game', 'fun travel game', 'strategy game', 'board games', 'game closet', 'game board', 'party game', '4 game', 'family board game', 'game changer', 'gameboard', 'gameplay', 'games', 'game experience', 'fun party game', 'travel game', 'family game', 'game night', 'party game race', 'game night', 'card game', 'family card game', 'word game', 'family game night', 'game guide', 'gameboard', 'fun game', 'picture guess game', 'education kid game', 'family board game', 'kids board game', 'question game', 'board games', 'card game', 'fun game', 'game board', 'twister game', 'group game', 'game night', 'game champ', 'strategy board game', 'game closet', 'game piece', 'game board', 'game set', 

In [8]:
most_frequent_games = filter_and_order_tags_by_frequency(df_with_collapsed_tags, "common_noun_phrases", "game")
print("Most frequent tags containing 'game':", most_frequent_games)

Most frequent tags containing 'game': [('card game', 66), ('board games', 60), ('board game', 49), ('strategy game', 48), ('family board game', 40), ('family game night', 37), ('family game', 26), ('party game', 25), ('strategy board game', 20), ('property trading game', 16), ('roleplay game', 14), ('fun game', 13), ('travel game', 11), ('kids board game', 11), ('group game', 10), ('murder mystery game', 10), ('player game', 9), ('adult game', 9), ('dice game', 8), ('family card game', 7), ('word game', 7), ('two game', 7), ('fun board game', 7), ('fun card game', 6), ('4 game', 5), ('war board game', 5), ('risk game', 5), ('kids game', 5), ('thrill strategy game', 5), ('role game', 5), ('bluff game', 5), ('sell game', 5), ('fantasy game', 5), ('thrilling game', 5), ('war game', 5), ('kid board game', 4), ('education kid game', 4), ('battle game', 4), ('kid card game', 4), ('4 player adventure game', 4), ('strategy card game', 4), ('base game', 4), ('escape room game', 4), ('kindergart

### Now Adding Frequent Games Tages back to Dataframe

In [9]:
game_tagged_df = add_game_tags_column(df_with_collapsed_tags, most_frequent_games, 
                                      common_noun_phrases_column= "common_noun_phrases")
# Conditionally fill 'game_tags' with ["game"] only if it is an empty list
game_tagged_df["game_tags"] = game_tagged_df["game_tags"].apply(lambda tags: ["game"] if tags == [] else tags)
game_tagged_df.drop(columns=["common_noun_phrases"], inplace=True)
game_tagged_df.head(10)

,reviews_count,price,rating,id,description,asin,title,sales_volume,game_tags
0,3c943db5-ad7e-45da-85a5-2363584335b1,B07MV4NN5Z,"Big Potato The Chameleon, Award-Winning Board ...",15.99,4.7,2K+ bought in past month,,7063,[game]
1,0ce92bc9-af3e-4e03-a362-75dfdac41f84,B09NYD15ZH,Skillmatics Rapid Rumble - Fast-Paced Board Ga...,19.97,4.6,1K+ bought in past month,,3147,[game]
2,44ae72ff-b3cb-45b2-b38f-a814e6a26d3e,B076HK9H7Z,"Sorry! Kids Board Game, Family Board Games for...",7.49,4.8,10K+ bought in past month,"Slide, collide and score to win the Sorry! gam...",31332,"[family board game, kid board game]"
3,99f3cd2b-7e42-4abd-a76f-32ae6855834a,B00D7OAOYI,"Hasbro Gaming Trouble Kids Board Game, Pop-o-M...",8.99,4.7,10K+ bought in past month,Product Description The Trouble game is racing...,37102,"[trouble gameplay, kid board game, family game..."
4,ceb3ba0f-3bdf-45eb-8d93-637369b6d069,B00000DMF5,Hasbro Gaming Candy Land Kingdom of Sweet Adve...,10.49,4.8,10K+ bought in past month,Product Description If you loved playing the C...,35049,[board game]
5,4ce2ba59-d635-4021-b5ae-037f4c2fc4fa,B00D8STBHY,"Hasbro Gaming Connect 4 Classic Grid,4 in a Ro...",8.99,4.8,9K+ bought in past month,Product Description Go 4 the win! The Connect ...,78198,"[player board game, fun travel game, strategy ..."
6,4030f2ef-db79-41c2-8bd0-5cb202da0c38,B0B5HLZ8T4,"Monopoly Game, Family Board Games for 2 to 6 P...",19.92,4.8,6K+ bought in past month,"Buy, sell, dream, and scheme the way to riches...",3579,[family board game]
7,3edbf10f-2cb6-4f54-ac45-46398d8096fb,B00ABA0ZOA,Jenga Game | The Original Wood Block Game with...,10.40,4.8,#1 Top Rated,Product Description Want a game experience tha...,100759,"[fun party game, travel game, family game]"
8,57234401-010f-42a9-b773-e98ef129cd4d,B077Z1R28P,Taco Cat Goat Cheese Pizza,9.84,4.8,10K+ bought in past month,This is the social card game that is going to ...,51819,"[party game race, card game]"
9,25e4faba-3811-4e1e-87b6-90fdae232236,B07P6MZPK3,Mattel Games UNO Card Game in a Collectible St...,10.99,4.8,10K+ bought in past month,UNO is the classic family card game that's eas...,71007,[family card game]


In [10]:
game_tagged_df.to_csv("Data/raw_data/optigame_products_with_tags.csv", index=False)